In [1]:
!pip install confluent-kafka==1.7.0

In [2]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [3]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [4]:
conf = {'bootstrap.servers': bootstrap_server}

In [5]:
a = AdminClient(conf)

In [6]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 4 topics:
  "SmokeSensorEvent" with 1 partition(s)
  "TemperatureSensorEvent" with 1 partition(s)
  "_schemas" with 1 partition(s)
  "__consumer_offsets" with 50 partition(s)


In [7]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *

import time

topic = "RoboticArm"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [8]:
producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

## run the following cell to loop across the data

they are the same data as those in the EPL example only the time flows at half of the speed

In [9]:
import json
from IPython.display import clear_output

def send(value):
    key = None
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    clear_output(wait=True)

while True:
    send({"id":"1", "status":"ready", "stressLevel": 0, "ts": int(time.time())})
    time.sleep(2)
    send({"id":"1", "status": "goodGrasped", "stressLevel": 1, "ts": int(time.time())})                                                        
    time.sleep(2)
    ts = int(time.time())
    send({"id":"1", "status":"movingGood", "stressLevel": 7, "ts": ts})
    send({"id":"2", "status":"ready", "stressLevel": 0, "ts": ts })
    time.sleep(2)
    send({"id":"2", "status":"goodGrasped", "stressLevel": 5, "ts":  int(time.time()) })
    time.sleep(1)
    send({"id":"2", "status":"movingGood", "stressLevel": 9, "ts":  int(time.time()) })
    time.sleep(10)
    ts = int(time.time())
    send({"id":"1", "status":"placingGood", "stressLevel": 3, "ts": ts})
    send({"id":"2", "status":"placingGood", "stressLevel": 3, "ts": ts })   
    time.sleep(8)
    ts = int(time.time())
    send({"id":"1", "status":"moving", "stressLevel": 2, "ts": ts})
    send({"id":"2", "status":"moving", "stressLevel": 1, "ts": ts })       
    time.sleep(6)
    ts = int(time.time())
    send({"id":"1", "status":"ready", "stressLevel": 0, "ts": ts})
    send({"id":"2", "status":"ready", "stressLevel": 0, "ts": ts })   
    time.sleep(2)

KeyboardInterrupt: 

to interrupt the execution of the cell, prese the square icon in the bar or choose *interrupt kernel* from the *kernel* dropdown menu